Again, make sure that the imported packages here are actually relevant to this section.

In [1]:
import requests
import urllib.request, urllib.parse
import json
from pprint import pprint
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
from time import time

url formats for genius API requests:

Song: https://api.genius.com/songs/[song_api_id]  
Artist: https://api.genius.com/artists/[artist_api_id]  
Search: https://api.genius.com/search?q=[search_term]

In [2]:
client_access_token = 'GNQKuuEfZCQ2zcB7axX9IXBr3Q_DQsm08Grhqz9UXx7vKz4yJYYMvLHHVYCALm0x'

In [5]:
# do i need this?

params = {'access_token':'GNQKuuEfZCQ2zcB7axX9IXBr3Q_DQsm08Grhqz9UXx7vKz4yJYYMvLHHVYCALm0x',
         'client_id':'Csa2S0VYHtT1lwuGTMaEmIt-2aa8gKh3386Hy7Ld-GGjhVTRam4wcgUfHJEpFmfb'}

In [6]:
full_song_dict_east = json.load(open('full_song_dict_east_HUGE.json'))
rappers_east = list(full_song_dict_east.keys())

full_song_dict_west = json.load(open('full_song_dict_west_HUGE.json'))
rappers_west = list(full_song_dict_west.keys())

In [1]:
##### WEST #####

In [ ]:
# have to explain why there's the counter --- why is it that sometimes the scrape doesn't work first time?

In [7]:
lyrics_west = {} # will collect lyrics with rappers as keys
errors_west = [] # collect errors- repeat scrape (where the request is valid but doesn't work) and investigate invalid
rogue_errors_west = [] # ?

# iterating over all (west coast) rapper names
for artist in tqdm(rappers_west):    
    
    # get the artist name in the correct format for genius urls (e.g. ).
    artist_search = re.sub("[.\(\)\']", '', artist).lower().capitalize().replace('&','and')
    artist_search = artist_search.replace('$','s').replace('-',' ').replace(' ','-')
    
    if artist == 'Paris':
        artist_search = 'Paris-rap' # an irregularity in the genius url formatting, due to common artist name
    
    try:
        art_lyr = []
        # iterating over each collection of song titles (album) for this artist
        for album in list(full_song_dict_west[artist].values()):

            for song in tqdm(album):
                
                # getting the song title into genius url format
                song_search = re.sub("[.\(\)\'\#\"\,\!\:\[\]]", '', song).lower().replace('&','and')
                while '  ' in song_search: 
                    song_search = song_search.replace('  ',' ')
                song_search = re.sub("[\-\:]", " ", song_search).replace('$','s').replace(' ','-')
                song_search = song_search.split('-feat')[0].split('-ft-')[0].split('feat-')[0]

                # using the re-formatted rapper and song names to create the url for the Genius lyrics scraping
                url = ('https://genius.com/' + artist_search + '-' + song_search + '-lyrics').replace('--','-')

                still_going = True # set to false once the lyrics have been successfully scraped
                count = 0 # counter for the number of attempts to scrape the lyrics to a particular song
                while still_going and (count < 4): 
                # once the scrape is successful or has been attempted 4 times, this while loop breaks.

                    try:
                        # pushing the request; using beautiful soup to find the lyrics within the html code
                        r = requests.get(url)
                        soup = BeautifulSoup(r.text, "html.parser")

                        lyr = soup.find("div", class_="lyrics").text
                        lyr = re.sub('\[.*?]','', lyr) # [Verse], [Bridge], etc. removed
                        lyr = re.sub('\n{2}','',lyr)  # Remove gaps between verses
                        lyr = str(lyr).strip('\n')
                        
                        art_lyr.append(lyr) # lyrics appended to a list of lyrics for this artist
                        still_going = False

                        
                    except ConnectionError:
                        
                        print('ConnectionError')
                        start = time()
                        while time() - start < 10:
                            continue
                        
                    except AttributeError as a:
                        # unsuccessful scrape (not due to my dodgy internet...)
                        count += 1

                        if count == 4:
                            errors_west.append([artist, song, song_search]) # store details of unsuccessful entries
                            print(f'attribute error:\n{a}')
                            
                    except Exception as e:
                        # storing other errors --- to investigate.
                        rogue_errors_west.append([artist, song, song_search])
                        print(f'Rogue exception: {e}')
                            
        lyrics_west[artist] = art_lyr  
            
    except:
        continue
        
        

 15%|█▌        | 3/20 [00:05<00:30,  1.82s/it]

attribute error:
'NoneType' object has no attribute 'text'



 20%|██        | 4/20 [00:06<00:27,  1.69s/it]

attribute error:
'NoneType' object has no attribute 'text'



100%|██████████| 20/20 [00:44<00:00,  2.23s/it]

 85%|████████▌ | 17/20 [00:39<00:06,  2.09s/it]

...


  5%|▌         | 1/20 [00:02<00:56,  2.97s/it]

attribute error:
'NoneType' object has no attribute 'text'



 60%|██████    | 12/20 [00:21<00:11,  1.48s/it]

attribute error:
'NoneType' object has no attribute 'text'



100%|██████████| 41/41 [2:01:29<00:00, 177.80s/it]


In [17]:
# check how many lyrics have been successfully scraped for each rapper

for artist in lyrics_west:
    
    songs = []
    for album in full_song_dict_west[artist]:
        songs.extend(full_song_dict_west[artist][album])
    
    print(artist, '---', len(lyrics_west[artist]), '---', len(songs))

2Pac --- 76 --- 84
Andre Nickatina --- 28 --- 46
Coolio --- 44 --- 49
Del the Funky Homosapien --- 35 --- 41
Digital Underground --- 50 --- 69
Dr. Dre --- 49 --- 56
Eazy-E --- 43 --- 54
House of Pain --- 43 --- 44
Ice Cube --- 66 --- 86
Luniz --- 22 --- 31
MC Eiht --- 39 --- 45
MC Hammer --- 36 --- 65
MC Ren --- 18 --- 34
N.W.A --- 29 --- 31
Paris --- 38 --- 55
People Under The Stairs --- 7 --- 16
Ras Kass --- 24 --- 31
Saafir --- 18 --- 46
The Pharcyde --- 30 --- 33
Kurupt --- 19 --- 20
Sir Mix-a-Lot --- 51 --- 69
Skee-Lo --- 10 --- 11
Souls of Mischief --- 28 --- 43
N.W.A. --- 30 --- 31
Too $hort --- 84 --- 93
Cypress Hill --- 56 --- 63
Kid Frost --- 11 --- 38
Warren G --- 63 --- 69
Compton's Most Wanted --- 38 --- 57
Nate Dogg --- 17 --- 20
Ice-T --- 79 --- 95
Westside Connection --- 9 --- 13
Blackalicious --- 16 --- 18
Aceyalone --- 13 --- 36
Tha Alkaholiks --- 37 --- 41
B-Legit --- 23 --- 42
Mack 10 --- 36 --- 48
Hieroglyphics --- 18 --- 20


In [20]:
# save the lyrics
with open('lyrics_west_HUGE.json', 'w') as fp:
    json.dump(lyrics_west, fp)

In [23]:
##### EAST ######

same code explanation.

In [24]:
lyrics_east = {}
errors_east = []
rogue_errors_east = []

global artist
for artist in tqdm(rappers_east):    
    
    artist_search = re.sub("[.\(\)\']", '', artist).lower().capitalize().replace('&','and')
    artist_search = artist_search.replace('$','s').replace('-',' ').replace(' ','-')

    
    try:
        art_lyr = []
        for album in list(full_song_dict_east[artist].values()):

            global song
            for song in tqdm(album):
                
                song_search = re.sub("[.\(\)\'\#\"\,\!\:\[\]]", '', song).lower().replace('&','and')
                while '  ' in song_search:
                    song_search = song_search.replace('  ',' ')
                song_search = re.sub("[\-\:]", " ", song_search).replace('$','s').replace(' ','-')
                song_search = song_search.split('-feat')[0].split('-ft-')[0].split('feat-')[0]


                url = ('https://genius.com/' + artist_search + '-' + song_search + '-lyrics').replace('--','-')

                still_going = True
                count = 0
                while still_going and (count < 4):

                    try:
                        r = requests.get(url)
                        soup = BeautifulSoup(r.text, "html.parser")

                        lyr = soup.find("div", class_="lyrics").text
                        lyr = re.sub('\[.*?]','', lyr)
                        lyr = re.sub('\n{2}','',lyr)
                        lyr = str(lyr).strip('\n')
                        
                        art_lyr.append(lyr)
                        still_going = False

                        
                    except ConnectionError:
                        
                        print('ConnectionError')
                        start = time()
                        while time() - start < 10:
                            continue
                        
                    except AttributeError as a:
                        count += 1

                        if count == 4:
                            errors_east.append([artist, song, song_search])
                            print(f'attribute error:\n{a}')
                            
                    except Exception as e:
                        rogue_errors_east.append([artist, song, song_search])
                        print(f'Rogue exception: {e}')
                            
        lyrics_east[artist] = art_lyr       
            
    except:
        continue

    

100%|██████████| 16/16 [00:44<00:00,  2.77s/it]

 25%|██▌       | 3/12 [00:05<00:18,  2.00s/it]

attribute error:
'NoneType' object has no attribute 'text'



 83%|████████▎ | 10/12 [00:18<00:03,  1.85s/it]

attribute error:
'NoneType' object has no attribute 'text'



 20%|██        | 2/10 [00:05<00:22,  2.81s/it]

...

attribute error:
'NoneType' object has no attribute 'text'



 92%|█████████▏| 11/12 [00:36<00:03,  3.40s/it]

attribute error:
'NoneType' object has no attribute 'text'



100%|██████████| 50/50 [1:54:02<00:00, 136.85s/it]

attribute error:
'NoneType' object has no attribute 'text'


In [26]:
for artist in lyrics_east:
    
    songs = []
    for album in full_song_dict_east[artist]:
        songs.extend(full_song_dict_east[artist][album])
    
    print(artist, '---', len(lyrics_east[artist]), '---', len(songs))

Cappadonna --- 26 --- 28
Grandmaster Caz --- 2 --- 10
Ghostface Killah --- 27 --- 29
GZA --- 27 --- 31
Inspectah Deck --- 16 --- 17
Kool Moe Dee --- 31 --- 35
KRS-One --- 36 --- 47
Big L --- 10 --- 12
Craig Mack --- 10 --- 24
Method Man --- 58 --- 71
Mos Def --- 16 --- 17
Nas --- 75 --- 90
The Notorious B.I.G. --- 53 --- 57
Ol' Dirty Bastard --- 43 --- 44
Pete Rock --- 14 --- 37
Raekwon --- 32 --- 38
Rakim --- 38 --- 47
RZA --- 19 --- 20
Eric B. & Rakim --- 49 --- 53
AZ --- 22 --- 26
Lauryn Hill --- 16 --- 16
Queen Latifah --- 40 --- 54
Wu-Tang Clan --- 26 --- 62
A Tribe Called Quest --- 73 --- 78
Mobb Deep --- 69 --- 81
Redman --- 69 --- 98
Gang Starr --- 73 --- 85
Shyheim --- 49 --- 72
Heltah Skeltah --- 26 --- 36
O.G.C. --- 24 --- 32
Black Moon --- 23 --- 25
Big Daddy Kane --- 85 --- 101
Fugees --- 32 --- 37
Smif-N-Wessun --- 15 --- 16
Brand Nubian --- 53 --- 66
Lord Finesse --- 24 --- 32
Showbiz and A.G. --- 33 --- 42
Das EFX --- 52 --- 57
EPMD --- 75 --- 83
Bahamadia --- 11 --- 14

In [27]:
with open('lyrics_east_HUGE.json', 'w') as fp:
    json.dump(lyrics_east, fp)

In [30]:
pd.DataFrame(errors_east).to_csv('errors_east_HUGE')

In [31]:
pd.DataFrame(errors_west).to_csv('errors_west_HUGE')

In [14]:
errors_eas = pd.read_csv('errors_east_HUGE').drop('Unnamed: 0', axis=1)
errors_wes = pd.read_csv('errors_west_HUGE').drop('Unnamed: 0', axis=1)

lyrics_east = json.load(open('lyrics_east_HUGE.json'))
lyrics_west = json.load(open('lyrics_west_HUGE.json'))

errors_east = [list(errors_eas.iloc[i].values) for i in range(len(errors_eas))]
errors_west = [list(errors_wes.iloc[i].values) for i in range(len(errors_wes))]

In [23]:
for artist in list(full_song_dict_west.keys()):
    
    if len(full_song_dict_west[artist]) == 0:
        print(artist)
        
# drop these from the west coast lyrics dictionary

Snoop Dogg
Tha Dogg Pound
mac Dre
Outlawz
Gift of Gab
Jurassic 5
Keak da Sneak
Big Syke
Shock G


In [24]:
for artist in list(full_song_dict_east.keys()):
    
    if len(full_song_dict_east[artist]) == 0:
        print(artist)
        
# likewise for the east coast lyrics dictionary

50 Cent
MF DOOM
Cormega
De La Soul
